# 🌐 **03 - LangGraph: Pipeline de Processamento**

## 🎯 **Objetivo:**
Implementar pipeline completo com LangGraph para processamento inteligente de requisições fitness.

## 📋 **O que faremos:**
1. 🏗️ Definir estrutura de estado (State)
2. 🔧 Implementar nós especializados
3. 🌐 Construir grafo do pipeline
4. 🧪 Testar fluxo completo

---

## 1️⃣ **Estrutura de Estado**

In [ ]:
# 📦 IMPORTS E CONFIGURAÇÕES

import json
import re
from typing import Dict, List, Any, TypedDict, Optional
from datetime import datetime
import hashlib

# Imports do LangGraph
try:
    from langgraph.graph import StateGraph, END
    from langgraph.checkpoint.memory import MemorySaver
    print("✅ LangGraph disponível")
    LANGGRAPH_AVAILABLE = True
except ImportError:
    print("⚠️ LangGraph não disponível - usando simulação")
    LANGGRAPH_AVAILABLE = False
    
    # Classes de fallback
    class StateGraph:
        def __init__(self, state_schema):
            self.nodes = {}
            self.edges = []
            self.state_schema = state_schema
        
        def add_node(self, name, func):
            self.nodes[name] = func
        
        def add_edge(self, from_node, to_node):
            self.edges.append((from_node, to_node))
        
        def add_conditional_edges(self, from_node, condition, mapping):
            pass
        
        def set_entry_point(self, node):
            self.entry_point = node
        
        def compile(self, checkpointer=None):
            return self
        
        def invoke(self, state, config=None):
            # Simulação básica
            current_state = state.copy()
            for node_name, node_func in self.nodes.items():
                try:
                    result = node_func(current_state)
                    if isinstance(result, dict):
                        current_state.update(result)
                except Exception as e:
                    print(f"❌ Erro no nó {node_name}: {e}")
            return current_state
    
    class MemorySaver:
        pass
    
    END = "__end__"

print("📦 Imports realizados!")

In [ ]:
# 🏗️ DEFINIÇÃO DO ESTADO

class FitnessState(TypedDict, total=False):
    """Estado compartilhado entre todos os nós do pipeline"""
    
    # Input do usuário
    pergunta_usuario: str
    objetivo: str
    periodicidade: int
    experiencia: str
    restricoes: List[str]
    
    # Dados processados
    pergunta_validada: bool
    dados_extraidos: Dict[str, Any]
    parametros_calculados: Dict[str, Any]
    
    # Contexto RAG
    contexto_rag: List[Dict[str, Any]]
    exercicios_disponiveis: Dict[str, List[str]]
    principios_treino: Dict[str, Any]
    
    # Busca externa
    precisa_busca_web: bool
    resultados_web: List[Dict[str, Any]]
    
    # Resultado final
    resposta_gerada: str
    plano_treino: Dict[str, Any]
    confianca: float
    
    # Metadados
    timestamp: str
    etapas_executadas: List[str]
    erros: List[str]

def criar_estado_inicial(pergunta: str) -> FitnessState:
    """Cria estado inicial para o pipeline"""
    return {
        'pergunta_usuario': pergunta,
        'objetivo': '',
        'periodicidade': 0,
        'experiencia': '',
        'restricoes': [],
        'pergunta_validada': False,
        'dados_extraidos': {},
        'parametros_calculados': {},
        'contexto_rag': [],
        'exercicios_disponiveis': {},
        'principios_treino': {},
        'precisa_busca_web': False,
        'resultados_web': [],
        'resposta_gerada': '',
        'plano_treino': {},
        'confianca': 0.0,
        'timestamp': datetime.now().isoformat(),
        'etapas_executadas': [],
        'erros': []
    }

print("✅ Estrutura de estado definida!")
print(f"📊 Campos do estado: {len(FitnessState.__annotations__)}")

## 2️⃣ **Nós Especializados do Pipeline**

In [ ]:
# 🔍 NÓ 1: VALIDAÇÃO E EXTRAÇÃO

def no_validacao(state: FitnessState) -> Dict[str, Any]:
    """Valida pergunta e extrai informações básicas"""
    
    pergunta = state['pergunta_usuario']
    print(f"🔍 Validando: {pergunta[:50]}...")
    
    # Padrões para extração
    objetivos_map = {
        'hipertrofia': ['hipertrofia', 'massa', 'musculo', 'ganhar peso', 'bulk'],
        'emagrecimento': ['emagrecer', 'perder peso', 'cutting', 'queimar gordura', 'definir'],
        'forca': ['força', 'forca', 'powerlifting', 'mais forte'],
        'condicionamento': ['condicionamento', 'resistencia', 'cardio', 'preparo fisico']
    }
    
    experiencias_map = {
        'iniciante': ['iniciante', 'começando', 'nunca fiz', 'primeiro'],
        'intermediario': ['intermediario', 'alguns meses', 'algum tempo'],
        'avancado': ['avançado', 'avancado', 'anos', 'experiente']
    }
    
    # Extrair objetivo
    objetivo_detectado = 'hipertrofia'  # padrão
    pergunta_lower = pergunta.lower()
    
    for obj, termos in objetivos_map.items():
        if any(termo in pergunta_lower for termo in termos):
            objetivo_detectado = obj
            break
    
    # Extrair periodicidade
    periodicidade_detectada = 4  # padrão
    
    # Buscar números + "vez", "dia", "x"
    match_periodicidade = re.search(r'(\d+)\s*(vez|dia|x)', pergunta_lower)
    if match_periodicidade:
        periodicidade_detectada = min(int(match_periodicidade.group(1)), 6)
    
    # Extrair experiência
    experiencia_detectada = 'intermediario'  # padrão
    
    for exp, termos in experiencias_map.items():
        if any(termo in pergunta_lower for termo in termos):
            experiencia_detectada = exp
            break
    
    # Extrair restrições
    restricoes_detectadas = []
    restricoes_patterns = {
        'lesao joelho': ['joelho', 'lesão joelho'],
        'lesao costas': ['costas', 'lombar', 'coluna'],
        'tempo limitado': ['pouco tempo', 'rapido', 'rápido'],
        'sem equipamento': ['casa', 'sem academia', 'sem equipamento']
    }
    
    for restricao, termos in restricoes_patterns.items():
        if any(termo in pergunta_lower for termo in termos):
            restricoes_detectadas.append(restricao)
    
    # Validação
    validacao_ok = len(pergunta.strip()) >= 10  # mínimo de caracteres
    
    dados_extraidos = {
        'objetivo_detectado': objetivo_detectado,
        'periodicidade_detectada': periodicidade_detectada,
        'experiencia_detectada': experiencia_detectada,
        'restricoes_detectadas': restricoes_detectadas,
        'confianca_extracao': 0.8 if validacao_ok else 0.3
    }
    
    print(f"   ✅ Objetivo: {objetivo_detectado}")
    print(f"   ✅ Periodicidade: {periodicidade_detectada}x/semana")
    print(f"   ✅ Experiência: {experiencia_detectada}")
    
    return {
        'pergunta_validada': validacao_ok,
        'objetivo': objetivo_detectado,
        'periodicidade': periodicidade_detectada,
        'experiencia': experiencia_detectada,
        'restricoes': restricoes_detectadas,
        'dados_extraidos': dados_extraidos,
        'etapas_executadas': state['etapas_executadas'] + ['validacao']
    }

print("✅ Nó de validação implementado!")

In [ ]:
# ⚙️ NÓ 2: CÁLCULOS E PARÂMETROS

def no_calculos(state: FitnessState) -> Dict[str, Any]:
    """Calcula parâmetros específicos do treino"""
    
    objetivo = state['objetivo']
    periodicidade = state['periodicidade']
    experiencia = state['experiencia']
    
    print(f"⚙️ Calculando parâmetros para {objetivo} ({experiencia})")
    
    # Parâmetros base por objetivo
    parametros_base = {
        'hipertrofia': {
            'series': 3, 'repeticoes_min': 8, 'repeticoes_max': 12,
            'descanso_min': 60, 'descanso_max': 90, 'intensidade': 75
        },
        'forca': {
            'series': 4, 'repeticoes_min': 1, 'repeticoes_max': 6,
            'descanso_min': 180, 'descanso_max': 300, 'intensidade': 85
        },
        'emagrecimento': {
            'series': 3, 'repeticoes_min': 12, 'repeticoes_max': 20,
            'descanso_min': 30, 'descanso_max': 60, 'intensidade': 65
        },
        'condicionamento': {
            'series': 4, 'repeticoes_min': 15, 'repeticoes_max': 25,
            'descanso_min': 15, 'descanso_max': 45, 'intensidade': 70
        }
    }
    
    # Ajustes por experiência
    multiplicadores_experiencia = {
        'iniciante': {'series': 0.8, 'intensidade': 0.8, 'volume': 0.7},
        'intermediario': {'series': 1.0, 'intensidade': 1.0, 'volume': 1.0},
        'avancado': {'series': 1.2, 'intensidade': 1.1, 'volume': 1.3}
    }
    
    # Obter parâmetros base
    params = parametros_base.get(objetivo, parametros_base['hipertrofia']).copy()
    mult = multiplicadores_experiencia.get(experiencia, multiplicadores_experiencia['intermediario'])
    
    # Aplicar ajustes
    params['series'] = max(2, int(params['series'] * mult['series']))
    params['intensidade'] = min(95, int(params['intensidade'] * mult['intensidade']))
    
    # Calcular volume semanal
    exercicios_por_treino = {
        2: 6, 3: 5, 4: 4, 5: 3, 6: 3
    }.get(periodicidade, 4)
    
    volume_semanal = periodicidade * exercicios_por_treino * params['series']
    volume_ajustado = int(volume_semanal * mult['volume'])
    
    # Divisão muscular
    divisoes = {
        2: ['Upper', 'Lower'],
        3: ['Push', 'Pull', 'Legs'],
        4: ['Peito/Tríceps', 'Costas/Bíceps', 'Ombros', 'Pernas'],
        5: ['Peito', 'Costas', 'Ombros', 'Braços', 'Pernas'],
        6: ['Peito', 'Costas', 'Ombros', 'Braços', 'Pernas', 'Core']
    }
    
    divisao_treino = divisoes.get(periodicidade, divisoes[4])
    
    # Ajustes por restrições
    restricoes_ajustes = {}
    for restricao in state['restricoes']:
        if 'tempo limitado' in restricao:
            restricoes_ajustes['tempo_treino'] = 45  # minutos
            params['descanso_max'] = min(params['descanso_max'], 60)
        elif 'lesao' in restricao:
            restricoes_ajustes['intensidade_reduzida'] = True
            params['intensidade'] = max(50, params['intensidade'] - 15)
    
    parametros_calculados = {
        'parametros_treino': params,
        'exercicios_por_treino': exercicios_por_treino,
        'volume_semanal': volume_ajustado,
        'divisao_treino': divisao_treino,
        'restricoes_ajustes': restricoes_ajustes,
        'tempo_estimado': restricoes_ajustes.get('tempo_treino', 60)
    }
    
    print(f"   ✅ Séries: {params['series']}")
    print(f"   ✅ Repetições: {params['repeticoes_min']}-{params['repeticoes_max']}")
    print(f"   ✅ Volume semanal: {volume_ajustado} séries")
    print(f"   ✅ Divisão: {', '.join(divisao_treino)}")
    
    return {
        'parametros_calculados': parametros_calculados,
        'etapas_executadas': state['etapas_executadas'] + ['calculos']
    }

print("✅ Nó de cálculos implementado!")

In [ ]:
# 🔍 NÓ 3: BUSCA RAG

def no_rag(state: FitnessState) -> Dict[str, Any]:
    """Busca contexto relevante na base de conhecimento"""
    
    objetivo = state['objetivo']
    periodicidade = state['periodicidade']
    experiencia = state['experiencia']
    
    print(f"🔍 Buscando contexto RAG para {objetivo}")
    
    # Simulação de busca RAG (normalmente viria do notebook 02)
    # Em implementação real, importaria a classe FitnessRAGSystem
    
    # Contextos simulados baseados no objetivo
    contextos_base = {
        'hipertrofia': [
            {
                'texto': '### HIPERTROFIA\nSéries: 3-4\nRepetições: 8-12\nDescanso: 60-90 segundos\nVolume alto, intensidade moderada.',
                'score': 0.95,
                'secao': 'Hipertrofia',
                'metodo': 'semantica'
            },
            {
                'texto': '**Progressão de carga**\nAumente 2.5-5kg quando conseguir fazer todas as séries no limite superior de repetições.',
                'score': 0.87,
                'secao': 'Progressão',
                'metodo': 'semantica'
            }
        ],
        'forca': [
            {
                'texto': '### FORÇA\nSéries: 3-5\nRepetições: 1-6\nDescanso: 2-5 minutos\nIntensidade alta, volume baixo.',
                'score': 0.92,
                'secao': 'Força',
                'metodo': 'semantica'
            }
        ],
        'emagrecimento': [
            {
                'texto': '### EMAGRECIMENTO\nSéries: 3-4\nRepetições: 12-20\nDescanso: 30-60 segundos\nAlto volume, descanso curto.',
                'score': 0.89,
                'secao': 'Emagrecimento',
                'metodo': 'semantica'
            }
        ]
    }
    
    # Exercícios por grupo muscular
    exercicios_disponiveis = {
        'peito': ['Supino reto', 'Supino inclinado', 'Crucifixo', 'Flexão', 'Supino declinado'],
        'costas': ['Puxada frontal', 'Remada curvada', 'Levantamento terra', 'Remada unilateral', 'Pull-up'],
        'ombros': ['Desenvolvimento militar', 'Elevação lateral', 'Elevação posterior', 'Remada alta'],
        'bracos': ['Rosca direta', 'Tríceps testa', 'Rosca martelo', 'Tríceps pulley', 'Rosca concentrada'],
        'pernas': ['Agachamento', 'Leg press', 'Stiff', 'Avanço', 'Extensão de pernas'],
        'core': ['Prancha', 'Abdominal', 'Russian twist', 'Mountain climber', 'Bicicleta']
    }
    
    # Princípios de treino
    principios_treino = {
        'sobrecarga_progressiva': 'Aumente gradualmente a intensidade ou volume',
        'especificidade': f'Treine de acordo com seu objetivo: {objetivo}',
        'recuperacao': f'Descanse {72 if experiencia == "iniciante" else 48}h entre treinos do mesmo grupo',
        'frequencia': f'{periodicidade}x por semana é adequado para seu nível'
    }
    
    # Selecionar contextos relevantes
    contextos_relevantes = contextos_base.get(objetivo, contextos_base['hipertrofia'])
    
    # Adicionar contexto sobre experiência
    if experiencia == 'iniciante':
        contextos_relevantes.append({
            'texto': 'INICIANTES: Foque em aprender a técnica correta. Comece com pesos leves.',
            'score': 0.80,
            'secao': 'Iniciantes',
            'metodo': 'regra'
        })
    elif experiencia == 'avancado':
        contextos_relevantes.append({
            'texto': 'AVANÇADOS: Use técnicas avançadas como drop sets, supersets, periodização.',
            'score': 0.85,
            'secao': 'Avançados',
            'metodo': 'regra'
        })
    
    print(f"   ✅ Contextos encontrados: {len(contextos_relevantes)}")
    print(f"   ✅ Exercícios disponíveis: {sum(len(ex) for ex in exercicios_disponiveis.values())}")
    print(f"   ✅ Princípios aplicados: {len(principios_treino)}")
    
    return {
        'contexto_rag': contextos_relevantes,
        'exercicios_disponiveis': exercicios_disponiveis,
        'principios_treino': principios_treino,
        'etapas_executadas': state['etapas_executadas'] + ['rag']
    }

print("✅ Nó RAG implementado!")

In [ ]:
# 🌐 NÓ 4: BUSCA WEB (Opcional)

def no_busca_web(state: FitnessState) -> Dict[str, Any]:
    """Busca informações complementares na web se necessário"""
    
    # Verificar se precisa de busca externa
    contexto_rag = state['contexto_rag']
    restricoes = state['restricoes']
    
    precisa_busca = (
        len(contexto_rag) < 2 or  # Pouco contexto encontrado
        any('sem equipamento' in r for r in restricoes) or  # Treino em casa
        any('lesao' in r for r in restricoes)  # Restrições médicas
    )
    
    resultados_web = []
    
    if precisa_busca:
        print(f"🌐 Buscando informações complementares na web")
        
        # Simulação de busca web
        # Em implementação real, usaria APIs como Tavily, SerpAPI, etc.
        
        objetivo = state['objetivo']
        
        # Resultados simulados
        if 'sem equipamento' in str(restricoes):
            resultados_web = [
                {
                    'titulo': 'Treino em Casa Sem Equipamentos',
                    'conteudo': 'Exercícios usando peso corporal: flexões, agachamentos, burpees, pranchas.',
                    'fonte': 'fitness.com',
                    'relevancia': 0.9
                },
                {
                    'titulo': 'HIIT para Emagrecimento em Casa',
                    'conteudo': 'Circuito de 20 minutos com exercícios funcionais de alta intensidade.',
                    'fonte': 'treino-casa.com',
                    'relevancia': 0.85
                }
            ]
        
        elif any('lesao' in r for r in restricoes):
            resultados_web = [
                {
                    'titulo': 'Exercícios para Lesão de Joelho',
                    'conteudo': 'Evite exercícios de impacto. Prefira leg press, cadeira extensora com amplitude reduzida.',
                    'fonte': 'fisioterapia.com',
                    'relevancia': 0.95
                }
            ]
        
        else:
            resultados_web = [
                {
                    'titulo': f'Treino de {objetivo.title()} Atualizado',
                    'conteudo': f'Novas técnicas e estudos sobre treino para {objetivo}.',
                    'fonte': 'ciencia-exercicio.com',
                    'relevancia': 0.75
                }
            ]
        
        print(f"   ✅ Encontrados {len(resultados_web)} resultados relevantes")
        
    else:
        print(f"💾 Contexto RAG suficiente, pulando busca web")
    
    return {
        'precisa_busca_web': precisa_busca,
        'resultados_web': resultados_web,
        'etapas_executadas': state['etapas_executadas'] + ['busca_web' if precisa_busca else 'busca_web_skip']
    }

print("✅ Nó de busca web implementado!")

In [ ]:
# 🎯 NÓ 5: GERAÇÃO DE RESPOSTA

def no_geracao(state: FitnessState) -> Dict[str, Any]:
    """Gera resposta final e plano de treino personalizado"""
    
    objetivo = state['objetivo']
    periodicidade = state['periodicidade']
    experiencia = state['experiencia']
    parametros = state['parametros_calculados']
    contexto_rag = state['contexto_rag']
    exercicios = state['exercicios_disponiveis']
    
    print(f"🎯 Gerando plano personalizado para {objetivo}")
    
    # Selecionar exercícios por divisão
    divisao_treino = parametros['divisao_treino']
    exercicios_por_treino = parametros['exercicios_por_treino']
    
    # Mapear divisão para grupos musculares
    mapeamento_divisao = {
        'Upper': ['peito', 'costas', 'ombros', 'bracos'],
        'Lower': ['pernas', 'core'],
        'Push': ['peito', 'ombros', 'bracos'],
        'Pull': ['costas', 'bracos'],
        'Legs': ['pernas', 'core'],
        'Peito/Tríceps': ['peito', 'bracos'],
        'Costas/Bíceps': ['costas', 'bracos'],
        'Peito': ['peito'],
        'Costas': ['costas'],
        'Ombros': ['ombros'],
        'Braços': ['bracos'],
        'Pernas': ['pernas'],
        'Core': ['core']
    }
    
    # Criar plano semanal
    plano_semanal = {}
    
    for i, dia_treino in enumerate(divisao_treino, 1):
        grupos_do_dia = mapeamento_divisao.get(dia_treino, ['peito'])
        exercicios_do_dia = []
        
        # Selecionar exercícios para cada grupo
        exercicios_por_grupo = max(1, exercicios_por_treino // len(grupos_do_dia))
        
        for grupo in grupos_do_dia:
            if grupo in exercicios:
                exercicios_grupo = exercicios[grupo][:exercicios_por_grupo]
                for exercicio in exercicios_grupo:
                    exercicios_do_dia.append({
                        'nome': exercicio,
                        'grupo': grupo,
                        'series': parametros['parametros_treino']['series'],
                        'repeticoes': f"{parametros['parametros_treino']['repeticoes_min']}-{parametros['parametros_treino']['repeticoes_max']}",
                        'descanso': f"{parametros['parametros_treino']['descanso_min']}-{parametros['parametros_treino']['descanso_max']}s",
                        'intensidade': f"{parametros['parametros_treino']['intensidade']}%"
                    })
        
        plano_semanal[f'Dia {i} - {dia_treino}'] = exercicios_do_dia
    
    # Gerar resposta textual
    resposta_partes = [
        f"# 🏋️‍♂️ **Plano de Treino Personalizado - {objetivo.title()}**\n",
        f"**👤 Perfil:** {experiencia.title()}\n",
        f"**📅 Frequência:** {periodicidade}x por semana\n",
        f"**⏱️ Duração estimada:** {parametros['tempo_estimado']} minutos por treino\n\n",
        "## 📊 **Parâmetros do Seu Treino:**\n",
        f"- **Séries:** {parametros['parametros_treino']['series']} por exercício\n",
        f"- **Repetições:** {parametros['parametros_treino']['repeticoes_min']}-{parametros['parametros_treino']['repeticoes_max']}\n",
        f"- **Descanso:** {parametros['parametros_treino']['descanso_min']}-{parametros['parametros_treino']['descanso_max']} segundos\n",
        f"- **Intensidade:** {parametros['parametros_treino']['intensidade']}% do seu máximo\n\n",
        "## 🗓️ **Divisão Semanal:**\n"
    ]
    
    # Adicionar detalhes dos treinos
    for dia, exercicios_dia in plano_semanal.items():
        resposta_partes.append(f"\n### **{dia}**\n")
        for i, ex in enumerate(exercicios_dia, 1):
            resposta_partes.append(
                f"{i}. **{ex['nome']}** ({ex['grupo'].title()})\n"
                f"   - {ex['series']} séries x {ex['repeticoes']} repetições\n"
                f"   - Descanso: {ex['descanso']} | Intensidade: {ex['intensidade']}\n"
            )
    
    # Adicionar dicas do contexto RAG
    if contexto_rag:
        resposta_partes.append("\n## 💡 **Dicas Importantes:**\n")
        for ctx in contexto_rag[:2]:  # Máximo 2 contextos
            if ctx['score'] > 0.7:  # Apenas contextos relevantes
                dica = ctx['texto'].replace('###', '').replace('**', '').strip()
                if len(dica) > 100:
                    dica = dica[:100] + "..."
                resposta_partes.append(f"- {dica}\n")
    
    # Adicionar informações sobre restrições
    if state['restricoes']:
        resposta_partes.append("\n## ⚠️ **Adaptações para suas Restrições:**\n")
        for restricao in state['restricoes']:
            if 'tempo limitado' in restricao:
                resposta_partes.append("- ⏰ Treino otimizado para tempo limitado\n")
            elif 'lesao' in restricao:
                resposta_partes.append(f"- 🩹 Adaptado para: {restricao}\n")
            elif 'sem equipamento' in restricao:
                resposta_partes.append("- 🏠 Exercícios adaptados para casa\n")
    
    resposta_final = "".join(resposta_partes)
    
    # Calcular confiança da resposta
    confianca = 0.7  # base
    if len(contexto_rag) >= 2:
        confianca += 0.1
    if state['pergunta_validada']:
        confianca += 0.1
    if experiencia != 'avancado':  # mais confiança para iniciantes/intermediários
        confianca += 0.1
    
    confianca = min(1.0, confianca)
    
    print(f"   ✅ Plano gerado: {len(plano_semanal)} dias de treino")
    print(f"   ✅ Exercícios totais: {sum(len(ex) for ex in plano_semanal.values())}")
    print(f"   ✅ Confiança: {confianca:.1%}")
    
    return {
        'resposta_gerada': resposta_final,
        'plano_treino': plano_semanal,
        'confianca': confianca,
        'etapas_executadas': state['etapas_executadas'] + ['geracao']
    }

print("✅ Nó de geração implementado!")

## 3️⃣ **Construção do Grafo LangGraph**

In [ ]:
# 🌐 CONSTRUÇÃO DO PIPELINE LANGGRAPH

def construir_pipeline_fitness() -> StateGraph:
    """Constrói o pipeline completo do AI Fitness Coach"""
    
    print("🏗️ Construindo pipeline LangGraph...")
    
    # Criar grafo com esquema de estado
    workflow = StateGraph(FitnessState)
    
    # Adicionar nós
    workflow.add_node("validacao", no_validacao)
    workflow.add_node("calculos", no_calculos)
    workflow.add_node("rag", no_rag)
    workflow.add_node("busca_web", no_busca_web)
    workflow.add_node("geracao", no_geracao)
    
    # Definir fluxo do pipeline
    workflow.set_entry_point("validacao")
    
    # Fluxo linear otimizado
    workflow.add_edge("validacao", "calculos")
    workflow.add_edge("calculos", "rag")
    workflow.add_edge("rag", "busca_web")
    workflow.add_edge("busca_web", "geracao")
    workflow.add_edge("geracao", END)
    
    # Função condicional para decisões (se necessário)
    def decidir_proximo_no(state: FitnessState) -> str:
        """Decide qual nó executar baseado no estado"""
        
        # Por enquanto, fluxo linear
        # Aqui poderia ter lógica condicional mais complexa
        
        etapas = state.get('etapas_executadas', [])
        
        if 'validacao' not in etapas:
            return 'validacao'
        elif 'calculos' not in etapas:
            return 'calculos'
        elif 'rag' not in etapas:
            return 'rag'
        elif 'busca_web' not in etapas and 'busca_web_skip' not in etapas:
            return 'busca_web'
        elif 'geracao' not in etapas:
            return 'geracao'
        else:
            return END
    
    # Compilar grafo
    if LANGGRAPH_AVAILABLE:
        # Com checkpointing para sessões persistentes
        memory = MemorySaver()
        pipeline = workflow.compile(checkpointer=memory)
    else:
        # Versão simulada
        pipeline = workflow.compile()
    
    print("✅ Pipeline construído com sucesso!")
    print(f"   📊 Nós: {len(workflow.nodes)}")
    print(f"   🔄 Fluxo: validacao → calculos → rag → busca_web → geracao")
    
    return pipeline

# Construir pipeline
fitness_pipeline = construir_pipeline_fitness()

print(f"\n🎉 Pipeline AI Fitness Coach pronto!")

## 4️⃣ **Teste Completo do Pipeline**

In [ ]:
# 🧪 TESTE COMPLETO DO PIPELINE

def testar_pipeline(pergunta: str, pipeline) -> Dict[str, Any]:
    """Testa o pipeline completo com uma pergunta"""
    
    print(f"🧪 TESTANDO PIPELINE COMPLETO")
    print(f"📝 Pergunta: {pergunta}")
    print("=" * 60)
    
    # Criar estado inicial
    estado_inicial = criar_estado_inicial(pergunta)
    
    try:
        # Executar pipeline
        if LANGGRAPH_AVAILABLE:
            # Execução real com LangGraph
            config = {"configurable": {"thread_id": "teste_1"}}
            resultado_final = pipeline.invoke(estado_inicial, config=config)
        else:
            # Simulação para teste
            resultado_final = pipeline.invoke(estado_inicial)
        
        # Análise dos resultados
        print("\n📊 RESULTADOS DA EXECUÇÃO:")
        print(f"   • Etapas executadas: {', '.join(resultado_final.get('etapas_executadas', []))}")
        print(f"   • Pergunta validada: {'✅' if resultado_final.get('pergunta_validada') else '❌'}")
        print(f"   • Objetivo detectado: {resultado_final.get('objetivo', 'N/A')}")
        print(f"   • Periodicidade: {resultado_final.get('periodicidade', 0)}x/semana")
        print(f"   • Experiência: {resultado_final.get('experiencia', 'N/A')}")
        print(f"   • Contextos RAG: {len(resultado_final.get('contexto_rag', []))}")
        print(f"   • Busca web realizada: {'✅' if resultado_final.get('precisa_busca_web') else '❌'}")
        print(f"   • Confiança: {resultado_final.get('confianca', 0):.1%}")
        
        # Mostrar plano gerado
        plano = resultado_final.get('plano_treino', {})
        if plano:
            print(f"\n💪 PLANO DE TREINO GERADO:")
            total_exercicios = 0
            for dia, exercicios in plano.items():
                print(f"   • {dia}: {len(exercicios)} exercícios")
                total_exercicios += len(exercicios)
            print(f"   • Total: {total_exercicios} exercícios na semana")
        
        # Mostrar resposta (primeiras linhas)
        resposta = resultado_final.get('resposta_gerada', '')
        if resposta:
            print(f"\n📝 RESPOSTA GERADA (preview):")
            linhas = resposta.split('\n')[:5]
            for linha in linhas:
                print(f"   {linha}")
            print(f"   ... ({len(resposta)} caracteres totais)")
        
        # Verificar erros
        erros = resultado_final.get('erros', [])
        if erros:
            print(f"\n❌ ERROS ENCONTRADOS:")
            for erro in erros:
                print(f"   • {erro}")
        
        return resultado_final
        
    except Exception as e:
        print(f"❌ Erro na execução do pipeline: {e}")
        return estado_inicial

# Cenários de teste
cenarios_teste = [
    "Quero um treino de hipertrofia para iniciante, 4 vezes por semana",
    "Preciso emagrecer, posso treinar 5x na semana mas tenho lesão no joelho",
    "Sou avançado e quero ganhar força, 3 dias por semana"
]

print("🎯 EXECUTANDO TESTES DO PIPELINE\n")

resultados_testes = []
for i, cenario in enumerate(cenarios_teste, 1):
    print(f"\n{i}️⃣ CENÁRIO {i}:")
    resultado = testar_pipeline(cenario, fitness_pipeline)
    resultados_testes.append(resultado)
    
    if i < len(cenarios_teste):
        print("\n" + "-" * 60)

print("\n" + "=" * 60)
print(f"✅ Todos os testes concluídos! ({len(resultados_testes)} cenários)")
print(f"📈 Taxa de sucesso: {sum(1 for r in resultados_testes if r.get('resposta_gerada')) / len(resultados_testes) * 100:.0f}%")

## 📊 **Resumo da Etapa 03**

### ✅ **Conquistas:**
- 🏗️ Estrutura de estado TypedDict completa
- 🔧 5 nós especializados implementados
- 🌐 Pipeline LangGraph funcional
- 🧪 Sistema de testes abrangente
- 📊 Monitoramento de execução completo

### 🎯 **Nós do Pipeline:**
1. **Validação**: Extrai objetivo, periodicidade, experiência
2. **Cálculos**: Determina parâmetros de treino personalizados
3. **RAG**: Busca contexto na base de conhecimento
4. **Busca Web**: Complementa com informações externas
5. **Geração**: Cria plano completo e resposta final

### 🔄 **Fluxo do Pipeline:**
```
Pergunta → Validação → Cálculos → RAG → Busca Web → Geração → Resposta
```

### 📈 **Próximos Passos:**
- **04-Pipeline.ipynb**: Integração e otimização completa
- **05-Sistema.ipynb**: Interface final e demonstrações

---

🌐 **Pipeline LangGraph fitness pronto para integração!** 🏋️‍♂️